**SQL Analytics Queries**

In [ ]:
--  Top 10 Most Common Damage Types by Vehicle Make

SELECT
  v.make,
  d.damage_type,
  COUNT(d.damage_type) AS occurrences
FROM damage_reports d
JOIN inspections i ON d.inspection_id = i.inspection_id
JOIN vehicles v ON i.vehicle_vin = v.vin
GROUP BY v.make, d.damage_type
ORDER BY occurrences DESC
LIMIT 10

In [ ]:
-- Inspector Efficiency Metrics (Inspections/Day + Avg Severity)

SELECT
  ins.inspector_id,
  insp.name,
  COUNT(DISTINCT ins.inspection_id) AS total_inspections,
  COUNT(DISTINCT TO_DATE(ins.inspection_date)) AS active_days,
  ROUND(COUNT(DISTINCT ins.inspection_id) / COUNT(DISTINCT TO_DATE(ins.inspection_date)), 2) AS inspections_per_day,
  ROUND(AVG(d.severity), 2) AS avg_severity,
  ROUND(AVG(d.image_count), 2) AS avg_images_per_report
FROM damage_reports d
JOIN inspections ins ON d.inspection_id = ins.inspection_id
JOIN inspectors insp ON ins.inspector_id = insp.inspector_id
GROUP BY ins.inspector_id, insp.name
ORDER BY inspections_per_day DESC

In [ ]:
-- Monthly Trending of Inspection Volume and Damage Costs

SELECT
  YEAR(ins.inspection_date) AS year,
  MONTH(ins.inspection_date) AS month,
  COUNT(DISTINCT ins.inspection_id) AS inspections,
  COUNT(d.report_id) AS reports,
  ROUND(SUM(d.estimated_cost), 2) AS total_cost,
  ROUND(AVG(d.estimated_cost), 2) AS avg_cost
FROM damage_reports d
JOIN inspections ins ON d.inspection_id = ins.inspection_id
GROUP BY YEAR(ins.inspection_date), MONTH(ins.inspection_date)
ORDER BY year, month

In [ ]:
--  Geographic Heatmap Data for Damage Severity

SELECT
  ROUND(ins.location_lat, 1) AS lat_bin,
  ROUND(ins.location_lon, 1) AS lon_bin,
  ROUND(AVG(d.severity), 2) AS avg_severity,
  COUNT(*) AS report_count
FROM damage_reports d
JOIN inspections ins ON d.inspection_id = ins.inspection_id
GROUP BY ROUND(ins.location_lat, 1), ROUND(ins.location_lon, 1)
ORDER BY avg_severity DESC


**Real-time Monitoring Queries**

In [ ]:
-- Live inspection count (last hour)

SELECT COUNT(*) AS live_inspection_count
FROM inspections
WHERE inspection_date >= NOW() - INTERVAL '1 hour'


In [ ]:
-- Queue depth by region


SELECT 
  FLOOR(location_lat) AS lat_bucket,
  FLOOR(location_lon) AS lon_bucket,
  COUNT(*) AS queue_depth
FROM inspections
WHERE status in ('in_progress', 'pending_review')
GROUP BY FLOOR(location_lat), FLOOR(location_lon)


In [ ]:
-- Failed inspection alerts

SELECT *
FROM inspections
WHERE status = 'failed'
AND inspection_date >= NOW() - INTERVAL '1 hour';
